# Attention Head Weights via Relevance Analysis

This notebook analyzes attention head weights in transformer models using Layer-wise Relevance Propagation (LRP). It visualizes and quantifies the importance of different attention heads across layers.

In [ ]:
!pip install lxt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.7 MB/s eta 0:00:00
   ━

In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoTokenizer
from transformers.models.qwen2 import modeling_qwen2
import matplotlib.pyplot as plt
import numpy as np

from lxt.efficient import monkey_patch
from datasets import load_dataset

from tqdm import tqdm

import random

import pickle

In [ ]:
monkey_patch(modeling_qwen2, verbose=True)

Patched Qwen2MLP
Patched Qwen2RMSNorm
Patched Dropout
Patched transformers.models.qwen2.modeling_qwen2


In [ ]:
def hook_attention_output(module, input, output):
    # Assuming the attention output is the second element in the output tuple
    if isinstance(output, tuple) and len(output) > 1:
        # save the attention output and make sure the gradient is also saved
        # This assumes the attention weights are the second element in the tuple
        attention_output = output[1]
        module.attention_output = attention_output
        module.attention_output.retain_grad() if module.attention_output.requires_grad else None

In [ ]:
wikitext_test = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1", split="test")

README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2-0.5B')

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
encodings = tokenizer("\n\n".join(wikitext_test["text"]), return_tensors="pt", add_special_tokens=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (299078 > 32768). Running this sequence through the model will result in indexing errors


In [ ]:
max_length = 512
stride = 32
seq_len = encodings.input_ids.size(1)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
model = modeling_qwen2.Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B', device_map='cuda', attn_implementation="eager")

# optional gradient checkpointing to save memory (2x forward pass)
model.train()
model.gradient_checkpointing_enable()

# deactive gradients on parameters to save memory
for param in model.parameters():
    param.requires_grad = False

# apply hooks
for layer in model.model.layers:
    # We need to hook the attention module within each layer
    layer.self_attn.register_forward_hook(hook_attention_output)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
prev_end_loc = 0

for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    input_embeds = model.get_input_embeddings()(input_ids)
    output_logits = model(inputs_embeds=input_embeds.requires_grad_(), use_cache=False, output_attentions=True).logits

    max_logits, max_indices = torch.max(output_logits[:, -1, :], dim=-1)
    max_logits.backward(max_logits)

    # Print predictions for sanity every now and then
    rnd_digit = random.randint(0, 9)
    if rnd_digit == 0:
        print("Prediction:", tokenizer.convert_ids_to_tokens(max_indices))

    # trace relevance through layers
    attention_head_weights = [[0 for i in range(model.model.layers[0].self_attn.attention_output.shape[1])] for j in range(len(model.model.layers))]
    for i, layer in enumerate(model.model.layers):
        # Extract attention weights from the hooked module
        if hasattr(layer.self_attn, 'attention_output') and layer.self_attn.attention_output is not None:
            # Assuming attention_output is (batch_size, num_heads, sequence_length, sequence_length)
            # We'll take the mean across heads and batch size for simplicity
            # Get the relevance scores corresponding to the attention neurons
            relevance = (layer.self_attn.attention_output * layer.self_attn.attention_output.grad).float()
            # Add the values to the list
            for batch in relevance:
              for j, head in enumerate(batch):
                # Get the sum of all the sequence_length x sequence_length relevance values
                tensor = head.float().sum()
                # Write the scalar value to heads
                attention_head_weights[i][j] += tensor.cpu().item()


        else:
            print("No attention output found for layer", i)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break




  0%|          | 4/9347 [00:02<1:34:30,  1.65it/s]

Prediction: ['Ġ"']


  0%|          | 18/9347 [00:09<1:10:38,  2.20it/s]

Prediction: ['Ġproduction']


  0%|          | 23/9347 [00:11<1:10:36,  2.20it/s]

...

100%|█████████▉| 9313/9347 [1:17:11<00:16,  2.01it/s]

Prediction: ['Ġ=']


100%|█████████▉| 9328/9347 [1:17:18<00:09,  2.00it/s]

Prediction: ['Ġthe']


100%|█████████▉| 9331/9347 [1:17:20<00:07,  2.01it/s]


In [ ]:
# Normalize the attention head weights for each layer
for layer_weights in attention_head_weights:
    # Calculate the sum of weights for the current layer
    layer_sum = sum(layer_weights)

    # Normalize the heads within the current layer
    for i in range(len(layer_weights)):
        # Avoid division by zero in case all weights in a layer are zero
        layer_weights[i] /= layer_sum if layer_sum != 0 else 1e-9

# Now attention_head_weights contains the weights normalized within each layer, summing to 1
print("Normalized relevance sccores for attention head (per layer, summing to 1):")
# Print the first few layers for brevity
for i in range(min(5, len(attention_head_weights))):
    print(f"Layer {i}: {attention_head_weights[i]}")

Normalized relevance sccores for attention head (per layer, summing to 1):
Layer 0: [4.898155355525982, -1.7533686966225193, 0.09259235343624675, -0.09020358372119933, -2.5843668410271343, -0.04915374517094803, 0.109813776149702, 0.5635006229721587, 0.6145668853866807, 0.8916467981615518, 1.6372899321784413, -1.254289710881199, 0.7543612497437555, -2.8305443961315193]
Layer 1: [0.1638085312827339, 0.04502290187976562, 0.17355573295381532, 0.05992532884222761, 0.010336444857623082, 0.19904195216233941, 0.27058694534161787, 0.011137689732813101, -0.12107711008000197, 0.053272848534028985, 0.04115709627531604, 0.01251695656010643, 0.04631599699645117, 0.03439868466116342]
Layer 2: [0.23670640014882582, -0.8997623373880836, 0.3358542508071992, -0.12726516854407618, 0.4575153671270552, 0.21724623575148094, 0.22899868650602642, 0.014467164517781189, 0.21584959576272872, 0.38474758680671295, 0.007233355038187466, 0.02952313423842479, 0.00887985957686048, -0.1099941303491234]
Layer 3: [0.09584

In [ ]:
# Download all obtained results. Then we'll call it a day.
# Save entire atteenion head weights file as a pkl file

with open('attention_head_weights.pkl', 'wb') as f:
    pickle.dump(attention_head_weights, f)